In [18]:
from dotenv import load_dotenv
import os
import json

# Load document and start session 

In [2]:
load_dotenv()
H2O_API_KEY = os.getenv("H2O_API_KEY") 
print(H2O_API_KEY)

sk-y51R9UjZuoMJydh1MOEBqnJlxa3DtpOtnwDKEI6slV0na7Cg


In [3]:
from h2ogpte import H2OGPTE

client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key= H2O_API_KEY,
)

# Create a new collection
collection_id = client.create_collection(
    name='First meeting transcirpt',
    description='Information related to the first group meeting',
)
print(collection_id)

Please install the correct version of H2OGPTE with `pip install h2ogpte==1.4.9`.
You can enable strict version checking by passing strict_version_check=True.
949b0c36-ad0e-4f39-9687-5c90695cfb6b


In [4]:
with open('meeting_mintues.txt', 'rb') as f:
    meeting_data1 = client.upload('meeting_mintues.txt', f)
client.ingest_uploads(collection_id, [meeting_data1,])

Job(id='79a0a71a-f63e-4c0b-9a25-7430622e166e', name='Adding documents', passed=1.0, failed=0.0, progress=1.0, completed=True, canceled=False, date=datetime.datetime(2024, 3, 28, 3, 59, 16, tzinfo=TzInfo(UTC)), kind=<JobKind.IngestUploadsJob: 'IngestUploadsJob'>, statuses=[JobStatus(id='f6f52470dca241bbb96b6ff5d2649e4a', status='Indexing done.'), JobStatus(id='36fea923bf79437c86bd632a60267d92', status='Collecting done.')], errors=[], last_update_date=datetime.datetime(2024, 3, 28, 3, 59, 28, tzinfo=TzInfo(UTC)), duration='12s')

In [5]:
chat_session_id = client.create_chat_session(collection_id)

In [6]:
llm_list = client.get_llms() # default use first elem, index[0]
for llm in llm_list:
    print(llm)

{'base_model': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'prompt_type': 'mixtral', 'prompt_dict': {'promptA': '<<SYS>>\nYou are an AI that follows instructions extremely well and as helpful as possible.\n<</SYS>>\n\n', 'promptB': '<<SYS>>\nYou are an AI that follows instructions extremely well and as helpful as possible.\n<</SYS>>\n\n', 'PreInstruct': '<s> [INST] ', 'PreInput': None, 'PreResponse': '[/INST]', 'terminate_response': ['[INST]', '</s>'], 'chat_sep': ' ', 'chat_turn_sep': '</s> ', 'humanstr': '[INST]', 'botstr': '[/INST]', 'generates_leading_space': False, 'system_prompt': 'You are an AI that follows instructions extremely well and as helpful as possible.', 'can_handle_system_prompt': True}, 'load_8bit': False, 'load_4bit': False, 'low_bit_mode': 1, 'load_half': False, 'use_flash_attention_2': False, 'load_gptq': '', 'load_awq': '', 'load_exllama': False, 'use_safetensors': False, 'revision': None, 'use_gpu_id': False, 'gpu_id': None, 'compile_model': None, 'use_cache': None,

In [6]:
def rag_chat(client, chat_session_id, main_prompt, system_prompt):
    with client.connect(chat_session_id) as session:
        reply = session.query(
            message=main_prompt,
            system_prompt=system_prompt,
            rag_config={
            "rag_type": "rag", # https://h2oai.github.io/h2ogpte/getting_started.html#advanced-controls-for-document-q-a
            },
            llm="h2oai/h2ogpt-4096-llama2-70b-chat",
        )
        return reply

# Summarizer and system prompt 

In [11]:
system_prompt = """
You are a Meeting Assistant for a software company. You will be given the meeting log in the form of audio, powerpoint 
or PDF. Your general duties include but are not limited to summarising important points during the meeting, taking note 
of everyone's deliverables and organising all information in a clear and concise form. Upon reading your output all
members should have a clear understanding of the meeting objectives, agenda, and outcomes.
"""

In [35]:
summarizer_prompt = """
Meeting Assistant Prompt Template:

Please retrieve the following details based on the meeting input:

1. Attendees: Who attended this meeting? Please list the names and email addresses of participants.

2. Meeting Details:
    Date and Time: When did the meeting take place?
    Duration: How long did the meeting last?

3. Meeting Purpose: 
    Objective: What is the primary objective or purpose of the meeting?
    Desired Outcome: What outcomes do you hope to achieve by the end of the meeting?

4. Meeting summary: Provide a concise summary of the entire meeting by listing out all the main highlights
                    such that participants not in the meeting will be able to understand what happened during the meeting.
    
5. Action Items:
    Deliverables: What is the distribution of workload discussed during the meeting?
    Assign: For each attendee, list out in detail all the work assigned to them, be as precise and detailed as possible.


Present points 1-5 in the following JSON format: 
{"Attendees": "", "Meeting Details": {"Date and Time":"", "Duration":""}, "Meeting Details": {"Objective":"", "Desired Outcome":""}, "Meeting summary": "", "Action Items": {"Deliverables":"", "Assign":""}} 
"""

In [36]:
meeting_summary_response = rag_chat(client, chat_session_id, summarizer_prompt, system_prompt)
# print(meeting_summary_response.content)

In [49]:
json_object = json.loads(meeting_summary_response.content)
Attendees = json_object['Attendees']
print(Attendees)
Meeting_Details = json_object['Meeting Details']
Date_and_Time = Meeting_Details['Date and Time']
Duration = Meeting_Details['Duration']
print(Meeting_Details)
print(Date_and_Time)
print(Duration)
Meeting_Purpose = json_object['Meeting Purpose']
Objective = Meeting_Purpose['Objective']
Desired_Outcome = Meeting_Purpose['Desired Outcome']
print(Meeting_Purpose)
print(Objective)
print(Desired_Outcome)
Meeting_summary = json_object['Meeting summary']
print(Meeting_summary)
Action_Items = json_object['Action Items']['Assign']
print(Action_Items)

['Jeremy_PH', 'Hannah Nga', 'Ryan_Edward', 'Chan Yi Ru Micole', 'Ben_CH', 'Eugene_YJ']
{'Date and Time': '22:17:11', 'Duration': '22:54:59'}
22:17:11
22:54:59
{'Objective': 'To discuss the development of a chatbot that can summarize meetings', 'Desired Outcome': 'To have a clear understanding of the meeting objectives, agenda, and outcomes'}
To discuss the development of a chatbot that can summarize meetings
To have a clear understanding of the meeting objectives, agenda, and outcomes
The meeting discussed the development of a chatbot that can summarize meetings. The chatbot will be able to summarize the meeting minutes and provide a concise summary of the main highlights. The meeting also discussed the distribution of workload and assigned tasks to each attendee.
[{'Name': 'Jeremy_PH', 'Tasks': ['Provide a summary of the meeting minutes']}, {'Name': 'Hannah Nga', 'Tasks': ['Assist in the development of the chatbot']}, {'Name': 'Ryan_Edward', 'Tasks': ['Develop the chatbot']}, {'Name':

# Sentiment Analysis

In [11]:
basic_sentiment_prompt = """
Sentiment Prompt Template:

Please retrieve the following details based on the meeting input:

1. Attendees: Who attended this meeting? Please list the names and email addresses of participants.
    
2. Action Items:
    Deliverables: What is the distribution of workload discussed during the meeting?
    Assign: For each attendee, list out in detail all the work assigned to them, be as precise and detailed as possible.
            For each work assigned arange them in order of importance

Present points 1 and 2 in the following JSON format: 
{"Attendees": "", "Action Items": ["Deliverables":"", "Assign":""]} 
"""

In [50]:
sentiment_prompt = """
Meeting Assistant Prompt Template:

Please retrieve the following details based on the meeting input:

1. Attendees: Who attended this meeting? Please list the names and email addresses of participants.

2. Meeting Details:
    Date and Time: When did the meeting take place?
    Duration: How long did the meeting last?

3. Meeting Purpose: 
    Objective: What is the primary objective or purpose of the meeting?
    Desired Outcome: What outcomes do you hope to achieve by the end of the meeting?

4. Meeting summary: Provide a concise summary of the entire meeting by listing out all the main highlights
                    such that participants not in the meeting will be able to understand what happened during the meeting.
    
5. Action Items:
    Deliverables: What is the distribution of workload discussed during the meeting?
    Assign: For each attendee, list out in detail all the work assigned to them, be as precise and detailed as possible. Rank the assignment by either 'Low', 'Medium', or 'High' prioriry.


Present points 1-5 in mandarin chinese language and the following JSON format: 
{"Attendees": "", "Meeting Details": {"Date and Time":"", "Duration":""}, "Meeting Details": {"Objective":"", "Desired Outcome":""}, "Meeting summary": "", "Action Items": {"Deliverables":"", "Assign":""}} 
"""

In [51]:
meeting_summary_response = rag_chat(client, chat_session_id, sentiment_prompt, system_prompt)
print(meeting_summary_response.content)

{
"Attendees": [
"Ben_CH",
"Eugene_YJ",
"Hannah Nga",
"Jeremy_PH",
"Ryan_Edward",
"Chan Yi Ru Micole"
],
"Meeting Details": {
"Date and Time": "22:18:02",
"Duration": "22:54:59"
},
"Meeting Purpose": {
"Objective": "To discuss the development of a chatbot for a software company",
"Desired Outcome": "To outline the key features and functionalities of the chatbot, and to assign tasks to team members for its development"
},
"Meeting Summary": [
"Discussion of the chatbot's purpose and objectives",
"Outline of the chatbot's key features and functionalities",
"Assignment of tasks to team members for the chatbot's development"
],
"Action Items": {
"Deliverables": "Development of a chatbot for a software company",
"Assign": [
{
"Name": "Ryan_Edward",
"Task": "Develop a summary of the meeting and distribute it to all team members",
"Priority": "High"
},
{
"Name": "Chan Yi Ru Micole",
"Task": "Develop a bare bones model of the chatbot and push it out by the next meeting",
"Priority": "High"
},


# Language translation 

In [59]:
translate = """
{
"Attendees": [
"Ben_CH",
"Eugene_YJ",
"Hannah Nga",
"Jeremy_PH",
"Ryan_Edward",
"Chan Yi Ru Micole"
],
"Meeting Details": {
"Date and Time": "22:18:02",
"Duration": "22:54:59"
},
"Meeting Purpose": {
"Objective": "To discuss the development of a chatbot for a software company",
"Desired Outcome": "To outline the key features and functionalities of the chatbot, and to assign tasks to team members for its development"
},
"Meeting Summary": [
"Discussion of the chatbot's purpose and objectives",
"Outline of the chatbot's key features and functionalities",
"Assignment of tasks to team members for the chatbot's development"
],
"Action Items": {
"Deliverables": "Development of a chatbot for a software company",
"Assign": [
{
"Name": "Ryan_Edward",
"Task": "Develop a summary of the meeting and distribute it to all team members",
"Priority": "High"
},
{
"Name": "Chan Yi Ru Micole",
"Task": "Develop a bare bones model of the chatbot and push it out by the next meeting",
"Priority": "High"
},
{
"Name": "Eugene_YJ",
"Task": "Assist in the development of the chatbot's language translation feature",
"Priority": "Medium"
},
{
"Name": "Hannah Nga",
"Task": "Assist in the development of the chatbot's sentiment analysis feature",
"Priority": "Medium"
},
{
"Name": "Jeremy_PH",
"Task": "Assist in the development of the chatbot's task assignment feature",
"Priority": "Medium"
}
]
}
}
"""


In [60]:
translation_prompt = """
Translate the input question to mandarin chinese langauge and output in the following JSON format:

{"参与者": "", "会议详细信息": {"日期和时间":"", "持续时间":""}, "会议目的": {"目标":"", "期望结果":""}, "会议总结": "", "行动项": {"交付物":"", "分配":""}} 
"""

In [61]:
llm = "h2oai/h2ogpt-4096-llama2-70b-chat"
answer = client.answer_question(question=translate, system_prompt= translation_prompt, llm=llm).content

In [62]:
print(answer)

{
"参与者": [
"Ben_CH",
"Eugene_YJ",
"Hannah Nga",
"Jeremy_PH",
"Ryan_Edward",
"Chan Yi Ru Micole"
],
"会议详细信息": {
"日期和时间": "22:18:02",
"持续时间": "22:54:59"
},
"会议目的": {
"目标": "讨论软件公司的 chatbot 开发",
"期望结果": "outline  chatbot 的主要特性和功能，并分配任务给团队成员进行其开发"
},
"会议总结": [
"讨论 chatbot 的目的和目标",
"Outline  chatbot 的主要特性和功能",
"分配任务给团队成员进行 chatbot 的开发"
],
"行动项": {
"交付物": "软件公司的 chatbot 开发",
"分配": [
{
"名称": "Ryan_Edward",
"任务": "编写会议纪要并分发给所有团队成员",
"优先级": "高"
},
{
"名称": "Chan Yi Ru Micole",
"任务": "开发 chatbot 的基础模型并在下一次会议前推出",
"优先级": "高"
},
{
"名称": "Eugene_YJ",
"任务": "协助 chatbot 的语言翻译功能开发",
"优先级": "中等"
},
{
"名称": "Hannah Nga",
"任务": "协助 chatbot 的情感分析功能开发",
"优先级": "中等"
},
{
"名称": "Jeremy_PH",
"任务": "协助 chatbot 的任务分配功能开发",
"优先级": "中等"
}
]
}
}


In [8]:
langauge_system_prompt = """ 
你是一家软件公司的会议助理。会议记录以音频、PowerPoint或PDF形式提供。
你的一般职责包括但不限于总结会议中的重要要点，记录每个人的交付物，并以清晰简洁的形式组织所有信息。
阅读你的输出后，所有成员都应清楚了解会议的目标、议程和结果。
"""

In [8]:
language_prompt = """ 
输出为中文
"""